In [1]:
import cv2
import pandas as pd
from datetime import datetime
from ultralytics import YOLO
import numpy as np

classNames = [
    'person', 'bicycle', 'car', 'motorcycle', 'airplane',
    'bus', 'train', 'truck', 'boat', 'traffic light',
    'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
    'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
    'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
    'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard',
    'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
    'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut',
    'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table',
    'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock',
    'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

model = YOLO('yolov8l.pt')
video_path = "input_video.mp4"

cap = cv2.VideoCapture(video_path)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter('outputvid.mp4', fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

ball_positions = []
timestamps = []

dribble_count = 0
ball_velocity = 0
font = cv2.FONT_HERSHEY_SIMPLEX

frame_count = 0
previous_y = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, conf=0.7)

    for r in results:
        boxes = r.boxes
        for box in boxes:
            c = int(box.cls[0])
            cc = classNames[c]
            x1, y1, x2, y2 = box.xyxy[0]

            if cc == "sports ball":

                ball_positions.append((int((x1 + x2) / 2), int((y1 + y2) / 2)))
                timestamps.append(datetime.now())

                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color=(0, 22, 255), thickness=3)



    # Calculate y-distance every 5 frames
    frame_count += 1
    if frame_count % 5 == 0:
        current_y = ball_positions[-1][1]  # Take the y-coordinate of the last recorded ball position
        y_distance = current_y - previous_y  # Calculate y-distance

        # Set a threshold for downward dribble detection
        threshold = 70

        if y_distance > threshold:
            dribble_count += 1

        previous_y = current_y

    # Calculate ball velocity
    if len(ball_positions) > 1:
        distance = np.linalg.norm(np.array(ball_positions[-1]) - np.array(ball_positions[-2]))
        time_diff = (timestamps[-1] - timestamps[-2]).total_seconds()
        ball_velocity = distance / time_diff if time_diff > 0 else 0

    # Calculate dribble frequency
    dribble_frequency = dribble_count / (timestamps[-1] - timestamps[0]).total_seconds() if len(timestamps) > 1 else 0

    cv2.putText(frame, f'Dribble Count: {dribble_count}', (50, 50), font, 0.65, (0, 255, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, f'Ball Velocity: {ball_velocity:.2f} pixels/second', (50, 100), font, 0.65, (0, 255, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, f'Dribble Frequency: {dribble_frequency:.2f} dribbles/second', (50, 150), font, 0.65, (0, 255, 0), 1, cv2.LINE_AA)
    
    output_video.write(frame) 
    
    cv2.imshow("Video with Bounding Boxes", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
output_video.release()
cv2.destroyAllWindows()

data = {'Timestamp': timestamps[:-1], 'Ball_Position': ball_positions[:-1]}
df = pd.DataFrame(data)

df.to_csv('csv/dribble_data.csv', index=False)

print(f"Number of dribbles detected: {dribble_count}")
print(f"Average Ball Velocity: {ball_velocity:.2f} pixels/second")
print(f"Dribble Frequency: {dribble_frequency:.2f} dribbles/second")

/tmp/ipykernel_9807/3084596762.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'

0: 640x384 1 person, 1 sports ball, 378.7ms
Speed: 4.1ms preprocess, 378.7ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)
qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/siddhi/anaconda3/envs/dribble/lib/python3.10/site-packages/cv2/qt/plugins"

0: 640x384 1 person, 1 sports ball, 360.4ms
Speed: 1.3ms preproc

Number of dribbles detected: 104
Average Ball Velocity: 122.07 pixels/second
Dribble Frequency: 0.18 dribbles/second
